<a href="https://colab.research.google.com/github/bxck75/A1_Colabs/blob/master/uLtImAtEsTaRtEr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
#@title Installs, Imports and define paths
#@markdown > choose local or gdrive als root filesystem
import os,subprocess

# system functions
def recursive_glob(treeroot, pattern):
  results = []
  for base, dirs, files in os.walk(treeroot):
    goodfiles = fnmatch.filter(files, pattern)
    results.extend(os.path.join(base, f) for f in goodfiles)
  return results

def cmdline(cmd,pr=False):
  result =[]
  f = os.popen(cmd)
  try:
    for line in f:
      if pr == True:
        print(line)
        result.append(line),
  finally:
    f.close()
  return result

# stinking default folder...poef!
cmdline('rm -r /content/sample_data')
cmdline('pip install gitpython')
# pick fs
filesystem = "local" #@param ["gdrive", "local"]
if filesystem == 'gdrive':
  # sync google drive
  from google.colab import drive
  import os
  # if drive is needed uncomment
  print('Google drive ',end='')
  drive.mount('/content/drive',force_remount=True)
  print('Root folder set to ',end='')
  root = '/content/drive/My drive/image_learning'
  print(root)
else:
  print('Root folder set to ',end='')
  root = '/content/image_learning'
  print(root) 

#shoot root
os.makedirs(root,exist_ok = True)
os.chdir(root)

# install missing lib
cmdline('pip install sklearn imageio tensorflow-gpu==2.0.0-alpha0')
cmdline('apt install tree')

# import the rest
import numpy as np
from PIL import Image
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D,MaxPooling2D
from keras.optimizers import SGD, RMSprop,adam
from keras.utils import np_utils
import theano, glob, shutil, fnmatch, time, cv2, itertools
from IPython.display import clear_output, display, Image
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from google.colab.patches import cv2_imshow
import ipywidgets as widgets
from subprocess import PIPE, Popen

# list of reps uncomment to install
reps=[
#         'EIGENREPS',
        'ytdl-org/youtube-dl',
        'mikf/gallery-dl',
        'bxck75/piss-ant-pix2pix',
#         'mikf/gallery-dl',
#         'bxck75/datasets',
#         'EIGENREPS',
#         'tjwei/Flappy-Turtle.',
#         'tjwei/fonttools',
#         'tjwei/blender3d_import_psk_psa',
#         'lllyasviel/sketchKeras',
#         'Mckinsey666/Anime-Face-Dataset',
#         'chenyuntc/pytorch-book',
#         'lllyasviel/style2paints',
#         'llSourcell/GANS-for-style-transfer',
#         'opencv/open_model_zoo',
#         'hindupuravinash/the-gan-zoo',
#         'corenel/GAN-Zoo',
#         'eriklindernoren/Keras-GAN',
#         'junyanz/CycleGAN',
#         'junyanz/pytorch-CycleGAN-and-pix2pix',
#         'junyanz/iGAN', #----> !wget http://efrosgans.eecs.berkeley.edu/iGAN/datasets/church_64.zip <----dataset 	outdoor_128.zip 	handbag_128.zip !!!
#         'martinarjovsky/WassersteinGAN',
#         'shaoanlu/faceswap-GAN',
#           'LantaoYu/SeqGAN',
#         'tjwei/GANotebooks',
#         'adeshpande3/Tensorflow-Programs-and-Tutorials',
#         'adeshpande3/Generative-Adversarial-Networks',
#         'diegoalejogm/gans',
#         'osh/KerasGAN',
#         'r9y9/gantts',
#         'jayleicn/animeGAN',
#         'jayleicn/ImageNet-Training',
#         'Zardinality/WGAN-tensorflow',
#         'timsainb/Tensorflow-MultiGPU-VAE-GAN',
#         'Larox/python-moviepy-meetup',
#         'tjwei/keras-yolo3',
#         'tensorflow/gan',
#         'tensorflow/moonlight'
#         'tensorflow/models',
#         'tensorflow/datasets',
#         'tensorflow/docs',
#         'mnicnc404/CartoonGan-tensorflow',
#         'Yijunmaverick/CartoonGAN-Test-Pytorch-Torch',
#         'keras-team/keras-contrib',
#         'mnicnc404/CartoonGan-tensorflow',
]



# Gitgo class
class GitGo():
  
  def __init__(self,repos,sub_repos=False,chdir=True,path='/content/'):
    self.sub_repo_list = []
    self.GitUsers=[]
    self.repos = repos
    self.chdir = chdir
    self.path = path
    os.makedirs(self.path, exist_ok = True)
    if 'help' in self.repos:
      self.help()
    self.install_reps()
    self.custom_reps_setup()
    if sub_repos == True:
      self.get_other_reps()

  def help(self):
    return "* pulls git rep and shows files \
            * returns root path for the repository \
            * Function needs repository <user>/<repository name> combination\
            * Switch chdir and define the rootpath for the repository\
            * Use : GitGo(<list of reps to install>, sub_repos=<True/False, chdir=<True/False>, path=<root path>)\
            "
  
  def install_reps(self):    
    for rep in self.repos:
      self.rep=rep.split('/')
      # change folder check
      if self.chdir ==True:
        #Switch to path
        os.chdir(self.path)
      # pull the git repo
      cmdline('git clone https://github.com/'+self.rep[0]+'/'+self.rep[1]+'.git',True)
      # Set the return value for rep rootpath
      self.PATH=self.path+self.rep[1]
    # show imported files
    os.system('ls ' + root)

  def custom_reps_setup(self):
    # custom stuff for CartoonGAN-tensorflow and keras-team/keras-contrib
    if 'keras-team/keras-contrib' in self.repos:
      os.chdir(self.path+'/keras-contrib')
      cmdline('python convert_to_tf_keras.py')
      cmdline('USE_TF_KERAS=1')
      cmdline('python setup.py install')
      import tensorflow as tf
      tf.__version__     
    # custom setup stuff for gallery-dl repo
    if 'mikf/gallery-dl' in self.repos:
      os.chdir(root+'/gallery-dl')
      cmdline("pip install -e . |grep 'succes'",True)
    # custom setup stuff for youtube-dl repo
    if 'ytdl-org/youtube-dl' in self.repos:
      os.chdir(root+'/youtube-dl') 
      cmdline("pip install -e . |grep 'succes'",True)      
    # switch backt to root
    os.chdir(self.path)
         
  def get_other_reps(self):          
      for r in self.repos:
        self.GUSER=r.split('/')[0]
        self.repo_name=r.split('/')[1]
        self.GitUsers.append(self.GUSER)
        !curl https://api.github.com/users/{self.GUSER}/repos?per_page=100 | grep -o 'git@[^"]*' > {root}/info.txt
        cmdline('cat '+root+"/info.txt |awk -F ':' '{print $2}'|awk -F '.' '{print $1}' > "+self.path+"/"+self.GUSER+"_repositories.txt",True)
        with open(root+'/info.txt','r') as f:
            for line in f:
              cline=line.split(':')[1].split('.')[0]
              self.sub_repo_list.append(cline),

      print(self.sub_repo_list)          

 

      def __repr__(self):
        return self.PATH

G=GitGo(reps,sub_repos=True,path=root)

loot = recursive_glob(root,'req*.txt')
# print(loot)
# define folders
input_folder=root+'/images_raw'
input_folder_resized=root+'/images_resized'
# make folders
os.makedirs(input_folder, exist_ok = True)
os.makedirs(input_folder_resized, exist_ok = True)
# count files
listing_raw=os.listdir(input_folder)
listing_resized=os.listdir(input_folder_resized)

print('All repos have are installed on the '+filesystem+' filesystem!')

# clear_output()

Google drive Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
Root folder set to /content/drive/My drive/image_learning


OSError: ignored

In [17]:
keyword='single+fish'

!gallery-dl -d {input_folder} https://flickr.com/search/?text={keyword} -r 500kb --verbose -R 1 --sleep 0.04

clear_output()

[gallery-dl][debug] Version 1.10.5-dev - Git HEAD: 776e9e0
[gallery-dl][debug] Python 3.6.8 - Linux-4.14.137+-x86_64-with-Ubuntu-18.04-bionic
[gallery-dl][debug] requests 2.21.0 - urllib3 1.24.3
[gallery-dl][debug] Starting DownloadJob for 'https://flickr.com/search/?text=single+fish'
[gallery-dl][debug] Updating urllib3 ciphers
[flickr][debug] Using api_key authentication
[flickr][debug] Using FlickrSearchExtractor for 'https://flickr.com/search/?text=single+fish'
[urllib3.connectionpool][debug] Starting new HTTPS connection (1): api.flickr.com:443
[urllib3.connectionpool][debug] https://api.flickr.com:443 "GET /services/rest/?text=single+fish&extras=description%2Cdate_upload%2Ctags%2Cviews%2Cmedia%2Curl_o%2Curl_k%2Curl_h%2Curl_l&page=1&method=flickr.photos.search&format=json&nojsoncallback=1&api_key=ac4fd7aa98585b9eee1ba761c209de68 HTTP/1.1" 200 None
[downloader.http][warning] Invalid rate limit ('500kb')
[urllib3.connectionpool][debug] Starting new HTTPS connection (1): live.staticf

KeyboardInterrupt: ignored

In [0]:
heap=input_folder+'/flickr/search/'+keyword
print(heap)
print(input_folder)
for file in glob.glob(heap+'/*'):
    shutil.move(file, input_folder)
    plt.show(file)

In [0]:
!rm -r sample_data
path1='/content/input_data'
'
!mkdir -p {path1}
!mkdir -p {path2}
!mv -r path1+'/sexcom/search/ecchi/' {path1}
path1+'/sexcom/search/ecchi')
num_samples= size(listing)
print(num_samples)

In [0]:
!cat /content/gallery-dl/docs/gallery-dl.conf

# !gallery-dl --list-extractors |grep 'http' > /content/gallery-dl/extractor_urls.txt
# !gallery-dl --help

# !youtube-dl --list-extractors |grep 'http' > /content/gallery-dl/extractor_urls.txt
# !youtube-dl --help

sage: gallery-dl [OPTION]... URL...

General Options:
  -h, --help                Print this help message and exit
  --version                 Print program version and exit
  -d, --dest DEST           Destination directory
  -i, --input-file FILE     Download URLs found in FILE ('-' for stdin)
  --cookies FILE            File to load additional cookies from
  --proxy URL               Use the specified proxy
  --clear-cache             Delete all cached login sessions, cookies, etc.

Output Options:
  -q, --quiet               Activate quiet mode
  -v, --verbose             Print various debugging information
  -g, --get-urls            Print URLs instead of downloading
  -j, --dump-json           Print JSON information
  -s, --simulate            Simulate data extraction; do not download anything
  -K, --list-keywords       Print a list of available keywords and example
                            values for the given URLs
  --list-modules            Print a list of available extractor modules
  --list-extractors         Print a list of extractor classes with
                            description, (sub)category and example URL
  --write-log FILE          Write logging output to FILE
  --write-unsupported FILE  Write URLs, which get emitted by other extractors
                            but cannot be handled, to FILE

Downloader Options:
  -r, --limit-rate RATE     Maximum download rate (e.g. 500k or 2.5M)
  -R, --retries N           Maximum number of retries for failed HTTP requests
                            or -1 for infinite retries (default: 4)
  -A, --abort N             Abort extractor run after N consecutive file
                            downloads have been skipped, e.g. if files with
                            the same filename already exist
  --http-timeout SECONDS    Timeout for HTTP connections (defaut: 30.0)
  --sleep SECONDS           Number of seconds to sleep before each download
  --no-part                 Do not use .part files
  --no-mtime                Do not set file modification times according to
                            Last-Modified HTTP response headers
  --no-download             Do not download any files
  --no-check-certificate    Disable HTTPS certificate validation

Configuration Options:
  -c, --config FILE         Additional configuration files
  -o, --option OPT          Additional '<key>=<value>' option values
  --ignore-config           Do not read the default configuration files

Authentication Options:
  -u, --username USER       Username to login with
  -p, --password PASS       Password belonging to the given username
  --netrc                   Enable .netrc authentication data

Selection Options:
  --download-archive FILE   Record all downloaded files in the archive file
                            and skip downloading any file already in it.
  --range RANGE             Index-range(s) specifying which images to
                            download. For example '5-10' or '1,3-5,10-'
  --chapter-range RANGE     Like '--range', but applies to manga-chapters and
                            other delegated URLs
  --filter EXPR             Python expression controlling which images to
                            download. Files for which the expression evaluates
                            to False are ignored. Available keys are the
                            filename-specific ones listed by '-K'. Example:
                            --filter "image_width >= 1000 and rating in ('s',
                            'q')"
  --chapter-filter EXPR     Like '--filter', but applies to manga-chapters and
                            other delegated URLs

Post-processing Options:
  --zip                     Store downloaded files in a ZIP archive
  --ugoira-conv             Convert Pixiv Ugoira to WebM (requires FFmpeg)
  --write-metadata          Write metadata to separate JSON files
  --write-tags              Write image tags to separate text files
  --mtime-from-date         Set file modification times according to 'date'
                            metadata